# CodeHem Quickstart (Python SDK)

This notebook demonstrates basic extraction, XPath queries, and safe patching using CodeHem as a library.

Install (if needed): `pip install CodeHem`

In [ ]:
from codehem import CodeHem, CodeElementType

code = '''\
class Greeter:
    def greet(self, name):
        print(f\"Hello, {name}\")
'''

hem = CodeHem('python')
elements = hem.extract(code)
[(e.type.value, e.name) for e in elements.elements]

## XPath queries
- Find class: `Greeter`
- Find method: `Greeter.greet`
- Request parts: `[def]` or `[body]`

In [ ]:
cls = CodeHem.filter(elements, 'Greeter')
method_body = hem.get_text_by_xpath(code, 'Greeter.greet[body]')
print('class:', cls.name)
print('body:
', method_body)

## Safe patch (dry-run diff)
Compute hash of the current fragment, preview a diff, then apply.

In [ ]:
frag_hash = hem.get_element_hash(code, 'Greeter.greet')
new_def = '''\
    def greet(self, name):
        print(f\"Hello, {name}!\")
        return 'ok'
'''
diff = hem.apply_patch(code, 'Greeter.greet', new_def, original_hash=frag_hash, dry_run=True)
print(diff.splitlines()[:10])  # show first lines

## Convert results to JSON
`model_dump()` (Pydantic v2) or `dict()` (v1) and exclude parser internals.

In [ ]:
payload = {
  'elements': [e.model_dump(exclude={'range': {'node'}}) if hasattr(e, 'model_dump') else e.dict(exclude={'range': {'node'}}) for e in elements.elements]
}
payload